In [ ]:
import socket
import threading

HOST = "127.0.0.1"
PORT = 9000

# filename -> set of (peer_host, peer_port)
file_index = {}
lock = threading.Lock()

def handle_peer(conn, addr):
    try:
        while True:
            data = conn.recv(4096).decode()
            if not data:
                break

            parts = data.split("|")
            command = parts[0]

            # REGISTER|peer_host|peer_port|file1,file2
            if command == "REGISTER":
                peer_host = parts[1]
                peer_port = parts[2]
                files = parts[3].split(",")

                with lock:
                    for f in files:
                        if f not in file_index:
                            file_index[f] = set()
                        file_index[f].add((peer_host, peer_port))

                print(f"[REGISTER] Peer {peer_host}:{peer_port} registered {files}")
                conn.sendall(b"REGISTERED")

            # SEARCH|filename
            elif command == "SEARCH":
                filename = parts[1]
                with lock:
                    peers = file_index.get(filename, set())

                response = ",".join([f"{h}:{p}" for h, p in peers])
                conn.sendall(response.encode())

                print(f"[SEARCH] {filename} -> {response}")

    except Exception as e:
        print("Peer error:", e)
    finally:
        conn.close()

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen()

    print(f"[INDEX SERVER] Running on {HOST}:{PORT}")

    while True:
        conn, addr = server.accept()
        threading.Thread(target=handle_peer, args=(conn, addr)).start()

if __name__ == "__main__":
    start_server()


In [ ]:
import socket
import threading
import os
import sys

INDEX_HOST = "127.0.0.1"
INDEX_PORT = 9000
BUFFER_SIZE = 4096

peer_host = "127.0.0.1"
peer_port = int(sys.argv[1])
shared_dir = sys.argv[2]

# ---------------- FILE SERVER ----------------
def file_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((peer_host, peer_port))
    server.listen()
    print(f"[PEER SERVER] Serving files on port {peer_port}")

    while True:
        conn, addr = server.accept()
        threading.Thread(target=send_file, args=(conn,)).start()

def send_file(conn):
    try:
        filename = conn.recv(BUFFER_SIZE).decode()
        filepath = os.path.join(shared_dir, filename)

        if not os.path.exists(filepath):
            conn.sendall(b"ERROR")
            return

        conn.sendall(b"OK")
        with open(filepath, "rb") as f:
            while chunk := f.read(BUFFER_SIZE):
                conn.sendall(chunk)

        print(f"[TRANSFER] Sent {filename}")
    finally:
        conn.close()

# ---------------- INDEX SERVER COMMUNICATION ----------------
def register_files():
    files = os.listdir(shared_dir)
    msg = f"REGISTER|{peer_host}|{peer_port}|{','.join(files)}"

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((INDEX_HOST, INDEX_PORT))
    s.sendall(msg.encode())
    print(s.recv(BUFFER_SIZE).decode())
    s.close()

def search_file(filename):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((INDEX_HOST, INDEX_PORT))
    s.sendall(f"SEARCH|{filename}".encode())
    response = s.recv(BUFFER_SIZE).decode()
    s.close()

    if not response:
        print("[SEARCH] File not found")
        return []

    peers = response.split(",")
    print(f"[SEARCH RESULT] {peers}")
    return peers

# ---------------- FILE DOWNLOAD ----------------
def download_file(peer, filename):
    host, port = peer.split(":")
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(filename.encode())

    status = s.recv(BUFFER_SIZE).decode()
    if status != "OK":
        print("[DOWNLOAD] File not available")
        s.close()
        return

    with open(os.path.join(shared_dir, filename), "wb") as f:
        while data := s.recv(BUFFER_SIZE):
            f.write(data)

    print(f"[DOWNLOAD] Received {filename}")
    s.close()

# ---------------- MAIN ----------------
if __name__ == "__main__":
    if not os.path.exists(shared_dir):
        os.makedirs(shared_dir)

    register_files()
    threading.Thread(target=file_server, daemon=True).start()

    while True:
        print("\n1. Search File\n2. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            fname = input("Enter filename: ")
            peers = search_file(fname)
            if peers:
                download_file(peers[0], fname)
        elif choice == "2":
            print("[PEER] Leaving network")
            break


In [ ]:
# Terminal 1 – Index Server
python index_server.py

# Terminal 2 – Peer 1
python peer.py 10001 shared1/

# Terminal 3 – Peer 2
python peer.py 10002 shared2/
